In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

movies = pd.read_excel('movie15.xls', skiprows=4)
def genre(x):
    if '성인물(에로)' not in x:
        return x
movies['장르'] = movies.장르.apply(genre)
movies = movies[~movies['장르'].isnull()].reset_index(drop=True)
print(len(movies.장르.unique()))
movies.head()

885


,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,이매큘레이트,Immaculate,2024.0,미국,장편,공포(호러),개봉,마이클 모한,NaN
2,러브 라이즈 블리딩,Love Lies Bleeding,2023.0,미국,장편,"범죄,멜로/로맨스",개봉,로즈 글래스,NaN
3,짱,Top Dog,2018.0,한국,장편,액션,개봉,송재덕,"(주)영화사 사계절,펀펀한영화사"
4,싸움의 기술 2,The Art of Fighting 2,2020.0,한국,장편,액션,개봉,김희성,(주)영화사 사계절


In [2]:
movies = movies.영화명.tolist()
movies[:15]

['부활의 노래',
 '이매큘레이트',
 '러브 라이즈 블리딩',
 '짱',
 '싸움의 기술 2',
 '아이윌 송',
 '싸움의 기술',
 '깡패들',
 '전야',
 '심장박동 조작극',
 '비포 선셋',
 '사일런트 나잇',
 '명탐정 코난: 100만 달러의 펜타그램',
 '그리고 바통은 넘겨졌다',
 '더 납작 엎드릴게요']

In [80]:
url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5
pd_movies = []
content = ''
cnt = 0
nums = list(range(10))

for movie in tqdm(movies):
    page = requests.get(url + movie)
    html = BeautifulSoup(page.text, 'html.parser')

    soup1 = html.select('#app div div div div') # 시놉시스
    soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

    evaluation = []
    st = 0
    for s in soup1:
        if s.text != '':
            if '시놉시스[편집]' in s.text:
                content = s.text
                break
    lst = content.split('[편집]')
    for i, v in enumerate(lst):
        if v[-4:] == '시놉시스':
            synopsys = lst[i+1]
            break
    for i, v in enumerate(soup1):
        if '평가[편집]' in v.text:
            evaluation.append(soup1[i:i+10])
    
    if evaluation == []: # (영화)라는 글자가 있어야 탐색이 되는 영화들도 존재하므로 데이터가 없을 땐 영화 라는 글자 추가
        page = requests.get(url + movie+'(영화)')
        html = BeautifulSoup(page.text, 'html.parser')

        soup1 = html.select('#app div div div div') # 시놉시스
        soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

        evaluation = []
        st = 0
        for s in soup1:
            if s.text != '':
                if '시놉시스[편집]' in s.text:
                    content = s.text
                    break
        lst = content.split('[편집]')
        for i, v in enumerate(lst):
            if v[-4:] == '시놉시스':
                synopsys = lst[i+1]
                break
        for i, v in enumerate(soup1):
            if '평가[편집]' in v.text:
                evaluation.append(soup1[i:i+10])

    if evaluation == []: # evaluation이 비어있으면 해당 영화의 데이터 수집을 패스
        pass
    else:
        evaluation = evaluation[-1]
        lst = [] # 영화에 대한 정보를 담는 리스트
        for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
            if s.text != '':
                if s.text == '장르':
                    st = 1
                elif '개요' in s.text:
                    st = 0
                    break
                if st == 1:
                    lst.append(s.text) # 영화 정보 담기
            if '개요' in s.text:
                break

        

        table_data = []
        data = []
        idx = -1
        if len(evaluation[idx]) == 0: # 페이지마다 스크래핑되는 경우와 안되는 경우가 있음.
            idx = -2                  # 인덱스를 바꾸면 가져올 수 있음

        try: # evaluation을 가져왔으나 속성값이 없어서 에러가 나오는 경우 => 패스
            for i in evaluation[idx].select('a'):
                if ('http' not in i.attrs['title']) & (i.attrs['title'] != '') & (len(i.attrs['title']) < 30):
                    try:
                        if '네이버 영화' in table_data[-1]: # 네이버 영화는 동일한 데이터가 2번 입력됨
                            del data[0]
                    except:
                        pass
                    table_data.append(data)
                    table_data.append(i.attrs['title'])
                    data = []
                if (i.text != '') & bool(re.search(r'[\d%/]', i.text)) & (len(data) < 2) & ~bool(re.search(r'[\[\]]', i.text)): 
                    data.append(i.text)
        


            diff = False # 데이터가 제대로 안담기는 경우 
            if table_data != []: # table_data가 없으면 패스
            # for i in table_data:
                for j in nums:
                    if j in table_data:
                        diff = True
                        break
                    # if diff == True:
                    #     break
                if diff != True:
                    for i in evaluation[2].select('a'):
                        if ('http' not in i.attrs['title']) & (i.attrs['title'] != '') & (len(i.attrs['title']) < 30):
                            try:
                                if '네이버 영화' in table_data[-1]: # 네이버 영화는 동일한 데이터가 2번 입력됨
                                    del data[0]
                            except:
                                pass
                            table_data.append(data)
                            table_data.append(i.attrs['title'])
                            data = []
                        if (i.text != '') & bool(re.search(r'[\d%/]', i.text)) & (len(data) < 2) & ~bool(re.search(r'[\[\]]', i.text)): 
                            data.append(i.text)



                # 평가 기관 및 점수 데이터 가공
                table_data = table_data[1:] # 첫째 데이터는 비어있음
                eval_dict = dict(zip(table_data[::2],table_data[1::2])) # dict화

                max_length = max(len(v) for v in eval_dict.values()) # 평가 챕터가 열려있지 않으면 에러 발생하므로 try문으로 묶음
                for k, v in eval_dict.items(): # 데이터 길이 맞추기
                    if len(v) < max_length:
                        eval_dict[k] = v + [np.nan] * (max_length - len(v))

                eval_df = pd.DataFrame(eval_dict) # 데이터프레임화
                eval_df = eval_df.dropna(axis=1, how='all') # 데이터가 전부 비어있으면 삭제
                eval_df = eval_df.transpose().reset_index()
                eval_df.columns = ['평가', '점수1', '점수2']

                # 영화 정보 가공
                lst.insert(0, '영화명')
                lst.insert(1, movie)
                lst.append('시놉시스')
                lst.append(synopsys)
                df = pd.DataFrame(lst)
                df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
                df.columns = ['kind', 'content']
                df.set_index(df.kind, inplace=True)
                df = df.drop('kind', axis=1)
                pd_movies.append([eval_df,df])
        except:
            pass

100%|██████████| 13306/13306 [1:51:00<00:00,  2.00it/s]  


In [75]:
table_data != []

False

In [81]:
len(pd_movies)

1731

In [83]:
pd_movies[1700][1]

,content
kind,
영화명,마다가스카 2
장르,애니메이션
각본/감독,"에릭 다넬, 톰 맥그라스"
출연,"벤 스틸러, 크리스 락, 데이비드 슈위머, 제이다 핀켓 스미스 외"
제작,"미레일 소리아, 마크 스위프트"
음악,한스 짐머
제공사,드림웍스
제작사,PDI/드림웍스
수입사,CJ ENM MOVIE


In [ ]:
import pickle

In [90]:
# 영화 스크래핑 데이터 저장
# def save_movies(data, filename):
with open('movies.pkl', 'wb') as f:
    pickle.dump(pd_movies, f)
# save_movies()

In [91]:
with open('movies.pkl', 'rb') as f:
    d = pickle.load(f)

In [94]:
d[1][1]

,content
kind,
영화명,러브 라이즈 블리딩
장르,"범죄, 멜로/로맨스"
감독,로즈 글래스[1]
각본,로즈 글래스베로니카 토필스카
제작,안드레아 콘웰올리버 캐스먼
출연,크리스틴 스튜어트케이티 오브라이언지나 말론안나 바리시니코프데이브 프랭코에드 해리스 外
촬영,벤 포즈먼
편집,마크 타운즈
음악,클린트 맨셀[2]


In [57]:
movies[:20]

['부활의 노래',
 '이매큘레이트',
 '러브 라이즈 블리딩',
 '짱',
 '싸움의 기술 2',
 '아이윌 송',
 '싸움의 기술',
 '깡패들',
 '전야',
 '심장박동 조작극',
 '비포 선셋',
 '사일런트 나잇',
 '명탐정 코난: 100만 달러의 펜타그램',
 '그리고 바통은 넘겨졌다',
 '더 납작 엎드릴게요',
 '탈출: 프로젝트 사일런스',
 '탈주',
 '핸섬가이즈',
 '하이재킹',
 '굿바이 크루얼 월드']

In [ ]:
['부활의 노래', # 안나옴
#  '이매큘레이트',
#  '러브 라이즈 블리딩',
#  '짱',
 '싸움의 기술 2', # 연도로 표시되므로 탐색 불가
 '아이윌 송', # 안나옴
 '싸움의 기술', # 시놉시스가 없어서 안됨
#  '깡패들',
 '전야', # 연도로 추가 입력돼야 함
 '심장박동 조작극', # 그냥 검색 결과가 없음
#  '비포 선셋',
 '사일런트 나잇', # 안나옴
 '명탐정 코난: 100만 달러의 펜타그램', # 하기에 로직이 다소 복잡해서 패스!
 '그리고 바통은 넘겨졌다', # 시놉시스가 없음
 '더 납작 엎드릴게요', # 평가가 없음
#  '탈출: 프로젝트 사일런스',
 '탈주', # 탐색이 잘 안됨;
 '핸섬가이즈',
 '하이재킹',
 '굿바이 크루얼 월드'
 '애프터썬',
 '카운트',
 '호밀밭의 반항아',
 '플라이 미 투 더 문',
 '드림팰리스',
 '막걸리가 알려줄거야 ',
 '타로',
 '우마무스메 프리티 더비 새로운 시대의 문',
 '빅샤크5: 80일간의 해저일주', # 테이블과 평가가 아예 없음
 '김씨표류기']

In [36]:
이매큘레이트, 러브 라이즈 블리딩, 짱, 깡패들

SyntaxError: invalid syntax (2245243919.py, line 1)

In [65]:
pd_movies[6][1]

,content
kind,
영화명,탈출: 프로젝트 사일런스
장르,"재난, 스릴러, 첩보, 액션, 드라마"
감독,김태곤
각본,"김태곤, 김용화, 박주석"
제작,김용화
출연,"이선균, 주지훈, 김희원 外"
촬영,홍경표
미술,한아름[1]
편집,"허선미, 이건문"


In [16]:
movies[:10]

['부활의 노래',
 '이매큘레이트',
 '러브 라이즈 블리딩',
 '짱',
 '싸움의 기술 2',
 '아이윌 송',
 '싸움의 기술',
 '깡패들',
 '전야',
 '심장박동 조작극']

In [17]:
pd_movies[2][1]

,content
kind,
영화명,깡패들
장르,액션
감독,소재현
각본,제작
출연,"박두식, 임이지"
촬영,최용순
음악,제작사
대한민국 행정구 속령 펀펀한 영화사,배급사
대한민국 행정구 속령 (주)영화사 사계절,개봉일
